In [1]:
import numpy as np
import csv

# Load data

In [2]:
fields = {
    'Marital status',
    'Application mode',
    'Application order',
    'Course',
    'Daytime/evening attendance\t',
    'Previous qualification',
    'Previous qualification (grade)',
    'Nacionality',
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    'Admission grade',
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder',
    'Age at enrollment',
    'International',
    'Curricular units 1st sem (credited)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (grade)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (grade)',
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate',
    'Inflation rate',
    'GDP',
}

def load_student_data(path_data='student-dropout/data.csv'):
    '''
    Load dataset and remove 'Enrolled' target
    '''
    data = []
    with open(path_data, encoding='utf-8-sig') as f_data:
        for datum in csv.DictReader(f_data, delimiter=';'):
            remove = False
            for field in list(datum.keys()):
                if field in fields and datum[field]:
                    datum[field] = float(datum[field])
                if field == 'Target':
                    if datum[field] == 'Enrolled':
                        remove = True
                    elif datum[field] == 'Dropout':
                        datum[field] = -1.
                    else: # 'Graduated'
                        datum[field] = 1.
            if not remove: 
                data.append(datum)
    return data

In [3]:
student_data = load_student_data()
print('Number of examples:', len(student_data))
print('A example:\n', student_data[100])

Number of examples: 3630
A example:
 {'Marital status': 1.0, 'Application mode': 17.0, 'Application order': 3.0, 'Course': 9238.0, 'Daytime/evening attendance\t': 1.0, 'Previous qualification': 1.0, 'Previous qualification (grade)': 131.0, 'Nacionality': 1.0, "Mother's qualification": 1.0, "Father's qualification": 39.0, "Mother's occupation": 5.0, "Father's occupation": 3.0, 'Admission grade': 122.6, 'Displaced': 1.0, 'Educational special needs': 0.0, 'Debtor': 0.0, 'Tuition fees up to date': 1.0, 'Gender': 0.0, 'Scholarship holder': 0.0, 'Age at enrollment': 18.0, 'International': 0.0, 'Curricular units 1st sem (credited)': 0.0, 'Curricular units 1st sem (enrolled)': 6.0, 'Curricular units 1st sem (evaluations)': 7.0, 'Curricular units 1st sem (approved)': 6.0, 'Curricular units 1st sem (grade)': 13.0, 'Curricular units 1st sem (without evaluations)': 0.0, 'Curricular units 2nd sem (credited)': 0.0, 'Curricular units 2nd sem (enrolled)': 6.0, 'Curricular units 2nd sem (evaluations)':

# Feature transforms

In [4]:
def std_vals(data, f):
    vals = [entry[f] for entry in data]
    avg = sum(vals) / len(vals)
    dev = [(entry[f] - avg)**2 for entry in data]
    sd = (sum(dev) / len(vals))**0.5
    return avg, sd

In [5]:
def standard(v, std):
    return [(v - std[0]) / std[1]]

In [6]:
def raw(x):
    return [x]

In [7]:
def one_hot(v, entries):
    vec = len(entries) * [0]
    vec[entries.index(v)] = 1
    return vec

# Preprocess data

In [8]:
features = [
    ('Marital status', one_hot),
    ('Application mode', one_hot),
    ('Application order', raw),
    ('Course', one_hot),
    ('Daytime/evening attendance\t', raw),
    ('Previous qualification', one_hot),
    ('Previous qualification (grade)', standard),
    ('Nacionality', one_hot),
    ("Mother's qualification", one_hot),
    ("Father's qualification", one_hot),
    ("Mother's occupation", one_hot),
    ("Father's occupation", one_hot),
    ('Admission grade', standard),
    ('Displaced', raw),
    ('Educational special needs', raw),
    ('Debtor', raw),
    ('Tuition fees up to date', raw),
    ('Gender', raw),
    ('Scholarship holder', raw),
    ('Age at enrollment', raw),
    ('International', raw),
    ('Curricular units 1st sem (credited)', raw),
    ('Curricular units 1st sem (enrolled)', raw),
    ('Curricular units 1st sem (evaluations)', raw),
    ('Curricular units 1st sem (approved)', raw),
    ('Curricular units 1st sem (grade)', raw),
    ('Curricular units 1st sem (without evaluations)', raw),
    ('Curricular units 2nd sem (credited)', raw),
    ('Curricular units 2nd sem (enrolled)', raw),
    ('Curricular units 2nd sem (evaluations)', raw),
    ('Curricular units 2nd sem (approved)', raw),
    ('Curricular units 2nd sem (grade)', raw),
    ('Curricular units 2nd sem (without evaluations)', raw),
    ('Unemployment rate', standard),
    ('Inflation rate', standard),
    ('GDP', standard),
]

In [9]:
def preprocess(data, features):
    features = [('Target', raw)] + features
    std = {f : std_vals(data, f) \
           for (f,phi) in features if phi == standard}
    entries = {f : list(set([entry[f] for entry in data])) \
               for (f, phi) in features if phi == one_hot} 
    print('Mean and Std:', std)
    print('Entries in one_hot field:', entries)
    
    findex = 0
    # Print the meaning of features
    for (f, phi) in features[1:]: # skip 'Target'
        if phi == standard:
            print(findex, f, 'std')
            findex += 1
        elif phi == one_hot:
            for entry in entries[f]:
                print(findex, f, entry, 'one_hot')
                findex += 1
        else:
            print(findex, f, 'raw')
            findex += 1

    vals = []
    for entry in data:
        phis = []
        for (f, phi) in features:
            if phi == standard:
                phis.extend(phi(entry[f], std[f]))
            elif phi == one_hot:
                phis.extend(phi(entry[f], entries[f]))
            else:
                phis.extend(phi(entry[f]))
        vals.append(np.array([phis])) # phis of shape (1,D)
    
    data = np.vstack(vals)
    np.random.seed(0)
    np.random.shuffle(data)
    return data[:, 1:], data[:, 0:1]


In [10]:
data, labels = preprocess(student_data, features)
print('\nData shape:', data.shape)
print('Labels shape:', labels.shape)

Mean and Std: {'Previous qualification (grade)': (132.92060606060494, 13.236549060750294), 'Admission grade': (127.29393939393924, 14.609282556743532), 'Unemployment rate': (11.63035812672188, 2.667284425747152), 'Inflation rate': (1.2315977961432394, 1.3847204104409045), 'GDP': (-0.009256198347107725, 2.259674553698473)}
Entries in one_hot field: {'Marital status': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0], 'Application mode': [1.0, 2.0, 5.0, 39.0, 7.0, 42.0, 43.0, 44.0, 10.0, 15.0, 16.0, 17.0, 18.0, 51.0, 53.0, 57.0, 26.0, 27.0], 'Course': [33.0, 9254.0, 9991.0, 9670.0, 9130.0, 171.0, 9003.0, 9773.0, 8014.0, 9070.0, 9085.0, 9556.0, 9238.0, 9147.0, 9500.0, 9853.0, 9119.0], 'Previous qualification': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 39.0, 40.0, 9.0, 10.0, 42.0, 12.0, 43.0, 38.0, 15.0, 14.0, 19.0], 'Nacionality': [1.0, 2.0, 6.0, 11.0, 13.0, 14.0, 17.0, 21.0, 22.0, 24.0, 25.0, 26.0, 41.0, 62.0, 100.0, 101.0, 103.0, 105.0, 109.0], "Mother's qualification": [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 9.0, 10.0, 11.0

# Decision Tree

In [11]:
def argmax(l, f):
    """
    Return the element in list l that gives highest value on f

    @param l: C{List} of items
    @param f: C{Procedure} that maps an item into a numeric score
    @returns: the element of C{l} that has the highest score
    """
    vals = [f(x) for x in l]
    return l[vals.index(max(vals))]

In [12]:
# Decision tree node class
class DTNode:
    N_THRESHOLD = 4 # don't split if node has fewer examples than this
    H_THRESHOLD = .01 # don't split if node has entropy less than this
    H_REDUCTION_THRESHOLD = .001 # don't split if entropy reduction is less than this
    index = 0

    def __init__(self, data=None, config=None):
        self.config = config
        if config != None:
            self.N_THRESHOLD = config[0]
            self.H_THRESHOLD = config[1]
            self.H_REDUCTION_THRESHOLD = config[2]
        
        DTNode.index += 1
        self.index = DTNode.index # node has unique number
        self.data = data
        self.prob = None
        if data is not None:
            self.n = float(data.shape[0]) # number of examples
            self.indices = range(data.shape[1] - 1) # feature indices
            self.set_h()

        self.splits = {}

        self.feat_id = None # feature index
        self.thres = None # threshold
        self.lchild = None # left child
        self.rchild = None # right child
        self.parent = None

    # Create split on feature 'i' at value 'th'
    def split(self, i, th):
        self.feat_id = i
        self.thres = th
        self.lchild = DTNode(self.data[self.data[:, i] < th], self.config)
        self.rchild = DTNode(self.data[self.data[:, i] >= th], self.config)
        self.splits[i].remove(th)

    # Evaluate candidate split by weighted average entropy
    def split_eval(self, i, th):
        lc = DTNode(self.data[self.data[:, i] < th], self.config)
        rc = DTNode(self.data[self.data[:, i] >= th], self.config)
        pl = lc.n / self.n
        pr = 1.0 - pl
        avgH = pl * lc.H + pr * rc.H
        return avgH, lc, rc
    
    # Entropy of class labels in this node, assumes 1, -1
    def set_h(self):
        b = .001
        npos = np.sum(self.data[:,-1] == 1) # count labels 1
        p = (npos + b) / (self.n + b + b)
        self.prob = p
        self.H = -p * np.log(p) - (1-p) * np.log(1-p)

    def build_tree(self):
        if self.H < self.H_THRESHOLD or self.n <= self.N_THRESHOLD:
            return
        # Find the best split
        (i, th, (h, lc, rc)) = argmax([(i, th, self.split_eval(i, th)) \
                                            for i in self.indices \
                                            for th in self.get_splits(i)],
                                        lambda x : -x[2][0]) # x = (a, b, (h, c, d))
        
        if (self.H - h) < self.H_REDUCTION_THRESHOLD:
            return
        # Recurse
        self.feat_id = i
        self.thres = th
        self.lchild = lc
        self.rchild = rc
        self.lchild.parent = self
        self.rchild.parent = self
        self.lchild.build_tree()
        self.rchild.build_tree()
    
    # Sort examples and return middle points between every two consecutive sampes
    def get_splits(self, i):
        if i not in self.splits:
            d = np.sort(np.unique(self.data[:,i]), axis=None)
            d1 = d[:-1]
            d2 = d[1:]
            self.splits[i] = (d1 + d2) / 2.0
        return self.splits[i]

    # Classify a data point
    def classify(self, x):
        if self.feat_id == None: # leaf node
            return self.prob
        elif x[self.feat_id] < self.thres:
            return self.lchild.classify(x) # go to left child
        else:
            return self.rchild.classify(x) # go to right child
        
    def display(self, depth=0, max_depth=3):
        if depth > max_depth:
            print(depth*'  ', 'Depth >', max_depth)
        if self.feat_id is None:
            print(depth*'  ', '=>', '%.2f'%self.prob, '[ n=', self.n, ']')
            return
        print(depth*'  ', 'Ft.', self.feat_id, '<', self.thres, '[ n=', self.n, ']')
        self.lchild.display(depth+1, max_depth)
        self.rchild.display(depth+1, max_depth)

In [17]:
class DecisionTree:
    def fit(self, X, Y, config=None):
        D = np.hstack([X,Y])
        self.root = DTNode(D, config)
        self.root.build_tree()
    def predict(self, X):
        pred = np.array([np.apply_along_axis(self.root.classify, 1, X)]).T - 0.5
        pred[pred >= 0] = 1
        pred[pred < 0] = -1
        return pred
    def display(self, depth=0, max_depth=3):
        self.root.display(depth, max_depth)

In [18]:
def accuracy(pred, labels):
    '''Prediction accuracy'''
    return np.mean(np.sign(pred * labels) > 0.0)

def dt_eval(X_train, Y_train, X_test, Y_test, max_depth=5, verbose=True, config=None):
    dt = DecisionTree()
    dt.fit(X_train, Y_train, config)
    pred_test = dt.predict(X_test)
    acc = accuracy(pred_test, Y_test)
    if verbose:
        dt.display(max_depth=max_depth)
        print('Test accuracy:', acc)
    return acc

In [19]:
def cross_validate(data, labels, k=10, verbose=True, config=None):
    indices = np.random.permutation(data.shape[0])
    X = data[indices,:]
    Y = labels[indices,:]
    s_data = np.array_split(X, k, axis=0)
    s_labels = np.array_split(Y, k, axis=0)
    score_sum = 0
    for i in range(k):
        data_train = np.concatenate(s_data[:i] + s_data[i+1:], axis=0)
        labels_train = np.concatenate(s_labels[:i] + s_labels[i+1:], axis=0)
        data_test = np.array(s_data[i])
        labels_test = np.array(s_labels[i])
        if verbose == "first_tree":
            if i == 0:
                score_sum += dt_eval(data_train, labels_train, data_test, labels_test, verbose = True, config = config)
            else:
                score_sum += dt_eval(data_train, labels_train, data_test, labels_test, verbose = False, config = config)
        else:
            score_sum += dt_eval(data_train, labels_train, data_test, labels_test, verbose = verbose, config = config)
    print('Cross validation accuracy:', score_sum/k)
    return score_sum/k

cross_validate(data, labels, k=5, verbose=False)

Cross validation accuracy: 0.8628099173553719


0.8628099173553719

# Evaluation

In [20]:
def test_eval(data, labels, pct=0.25):
    '''
    pct - percentage for test set
    '''
    X = data
    Y = labels
    (n, d) = X.shape
    indices = np.random.permutation(n)  # randomize the data set
    tx = int((1-pct)*n)                 # size of training split
    training_idx, test_idx = indices[:tx], indices[tx:]
    trainingX, testX = X[training_idx,:], X[test_idx,:]
    trainingY, testY = Y[training_idx,:], Y[test_idx,:]
    return dt_eval(trainingX, trainingY, testX, testY)

test_eval(data, labels)

 Ft. 232 < 3.5 [ n= 2722.0 ]
   Ft. 224 < 0.5 [ n= 887.0 ]
     Ft. 214 < 0.903949975282321 [ n= 120.0 ]
       Ft. 218 < 0.5 [ n= 78.0 ]
         Ft. 136 < 0.5 [ n= 16.0 ]
           => 0.00 [ n= 14.0 ]
           => 0.50 [ n= 2.0 ]
         Ft. 60 < 0.04754970019046503 [ n= 62.0 ]
           Ft. 182 < 0.5 [ n= 32.0 ]
             Depth > 5
             Ft. 235 < -0.573751382473281 [ n= 30.0 ]
               Depth > 5
               Ft. 60 < -1.0894536026286241 [ n= 12.0 ]
                 Depth > 5
                 Ft. 9 < 0.5 [ n= 6.0 ]
                   Depth > 5
                   => 1.00 [ n= 3.0 ]
                   Depth > 5
                   => 0.33 [ n= 3.0 ]
                 Depth > 5
                 Ft. 60 < -0.33396968048969433 [ n= 6.0 ]
                   Depth > 5
                   => 0.00 [ n= 5.0 ]
                   Depth > 5
                   => 1.00 [ n= 1.0 ]
               Depth > 5
               Ft. 24 < 2.0 [ n= 18.0 ]
                 Depth > 5
         

0.8645374449339207